In [1]:
# Imports
import argparse
import os
import random
import json
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from accelerate import Accelerator, DeepSpeedPlugin
from accelerate import DistributedDataParallelKwargs
from models import DBLM2
from data_provider.data_loader import Dataset_Pred, get_result_list2
from utils.losses import (
    cvx_layer, cpu_compute_dfl_loss
)
from utils.infer_tool import (
    group_data_by_month_new, load_data, calculate_metrics, plot_cumulative_return
)
from utils.tools import del_files, EarlyStopping, adjust_learning_rate, vali
from utils.main_analysis import *
# Matplotlib settings
%matplotlib inline
import matplotlib.pyplot as plt


plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

# Torch and environment settings
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
os.environ['CURL_CA_BUNDLE'] = ''
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

# Set random seed
fix_seed = 2024
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)



def update_args(args):
    """
    Updates the args object with values from the shell script.
    All updates are based on the provided shell script variables.
    """
    shell_vars = {
        'HF_HOME': '/home/work/DBLM/huggingface_cache',
        'model_name': 'DBLM2',
        'train_epochs': 1,
        'learning_rate': 0.01,
        'llama_layers': 32,
        'master_port': 97,  # master_port in shell script is 00097, updated to integer 97
        'num_process': 4,
        'batch_size': 16,
        'comment': 'DBLM-SNP'
    }

    # Update environment variables if needed
    os.environ['HF_HOME'] = shell_vars['HF_HOME']

    # Update args with shell script variables
    args.model = shell_vars['model_name']
    args.train_epochs = shell_vars['train_epochs']
    args.learning_rate = shell_vars['learning_rate']
    args.llm_layers = shell_vars['llama_layers']
    args.num_workers = shell_vars['num_process']
    args.batch_size = shell_vars['batch_size']
    args.des = 'Exp'  # As per shell script
    args.model_id = 'SNP100_ret_1129'       # Updated from shell script
    args.data_path = 'SNP100_ret_1129.csv'  # Updated from shell script
    args.seq_len = 252  # As per shell script
    args.label_len = 0  # As per shell script
    args.pred_len = 22  # As per shell script
    args.d_ff = 36
    args.num_hidden = 512
    args.enc_in = 50  # As per shell script
    args.dec_in = 50  # As per shell script
    args.loss_alpha = 0.4
    args.lambda_value = "B"
    args.num_heads = 4
    args.num_enc_l = 1
    args.comment = shell_vars['comment']
    return args
parser = argparse.ArgumentParser(description='DBLM')

# Basic config
parser.add_argument('--task_name', type=str , default='Forecasting', help='task name, options:[DFL, Forecasting]')
parser.add_argument('--is_training', type=int, default=1, help='status')
parser.add_argument('--model_id', type=str, default='test', help='model id')
parser.add_argument('--model_comment', type=str,  default='none', help='prefix when saving test results')
parser.add_argument('--model', type=str, default='Autoformer', help='model name, options: [Autoformer, DLinear]')
parser.add_argument('--seed', type=int, default=2024, help='random seed')

# Data loader
parser.add_argument('--data', type=str, default='SNP_Multi', help='dataset type')
parser.add_argument('--root_path', type=str, default='./preprocessing', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='SNP100_ret_1129.csv', help='data file')
parser.add_argument('--loader', type=str, default='modal', help='dataset type')  # CHECK
parser.add_argument('--freq', type=str, default='d',
                    help='freq for time features encoding, '
                         'options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], '
                         'you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')
parser.add_argument('--is_price', type=bool, default=False, help='Use price data?')

# Forecasting task
parser.add_argument('--seq_len', type=int, default=252, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=22, help='prediction sequence length')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; '
                         'M:multivariate predict multivariate, S: univariate predict univariate, '
                         'MS:multivariate predict univariate')
# View define
parser.add_argument('--num_heads', type=int, default=4, help='number of heads')
parser.add_argument('--num_enc_l', type=int, default=1, help='number of encoder layers')
parser.add_argument('--loss_alpha', type=float, default=0.4, help='Loss function alpha')

# Model define
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--enc_in', type=int, default=50, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=50, help='decoder input size')
parser.add_argument('--num_hidden', type=int, default=256, help='number of hidden units')
parser.add_argument('--embed', type=str, default='timeF', help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')
parser.add_argument('--llm_model', type=str, default='GPT2', help='LLM model')  # LLAMA, GPT2, BERT
parser.add_argument('--llm_dim', type=int, default=768, help='LLM model dimension')  # LLama7b:4096; GPT2-small:768; BERT-base:768
parser.add_argument('--lambda_value', type=str, default='B', help='Lambda e.g.("HA", "A", "B", "C", "HC")')

# Prompt
parser.add_argument('--ticker_dict_path', type=str, default='./view_bank/ticker_dict2.json', help='Path to the ticker dictionary JSON file')
parser.add_argument('--sector_dict_path', type=str, default='./view_bank/sector_dict2.json', help='Path to the ticker dictionary JSON file')
parser.add_argument('--global_market_view', type=str, default='./view_bank/global_market_views.json',  help='Market view')
parser.add_argument('--global_stock_view',  type=str, default='./view_bank/global_stocks_views.json',  help='Stock view')
parser.add_argument('--global_user_view',   type=str, default=None,   help='User portfolio view')

parser.add_argument('--global_market_view_val', type=str, default='./view_bank/market_view_val.json',  help='Market view validation')
parser.add_argument('--global_stock_view_val',  type=str, default='./view_bank/stock_view_val.json',  help='Stock view validation')
parser.add_argument('--triplet_loss_weight', type=float, default=1.0, help='Weight for the triplet loss')
parser.add_argument('--d_ff', type=int, default=64, help='Dimension of feedforward network')

# Optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--align_epochs', type=int, default=10, help='alignment epochs')
parser.add_argument('--batch_size', type=int, default=16, help='batch size of train input data')
parser.add_argument('--eval_batch_size', type=int, default=8, help='batch size of model evaluation')
parser.add_argument('--patience', type=int, default=10, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='MSE', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')  # type1 COS
parser.add_argument('--pct_start', type=float, default=0.2, help='pct_start')  # Check
parser.add_argument('--use_amp', action='store_true', default=False, help='use automatic mixed precision training')
parser.add_argument('--llm_layers', type=int, default=6)

args = parser.parse_args([])

# Update args with shell script variables
args = update_args(args)

### ret

In [2]:
noise_values = [0.0543, 0.5690, 0.9545, 2.4305, 3.4623]

for risk in ["B"]: #"HA","A",,"C","HC"]:    

    # 실험 실행
    for loss in [4]:
        for head in [2]:
            for encoder in [1]:
                for dff in [12]:
                    for hidden in [128]:
                        for ii in [0]:
                            try:
                                model_name_template = f"mdDBLM2_daSNP_Multi_sq252_pr22_lo{loss}_he{head}_en{encoder}_llGPT2_la{risk}_deExp_is16_prFalse_dff{dff}_hi{hidden}_ii{ii}_Noise_None2-SNP"

                                weight_lists = run_experiment(args, loss, head, encoder, dff, hidden, noise_values, model_name_template, risk,ii)
                                try:
                                    process_results(weight_lists, f"loss={loss}, head={head}, num_enc_l={encoder}, dff={dff}, num_hidden={hidden}, ii={ii}", loss, head, encoder, dff, hidden, risk, ii)
                                except Exception as e:
                                    print(f"Error processing results for loss={loss}, head={head}, num_enc_l={encoder}, dff={dff}, num_hidden={hidden}, risk={risk}, ii={ii}: {e}")
                                    pass
                            except Exception as e:
                                print(f"Error running experiment for loss={loss}, head={head}, num_enc_l={encoder}, dff={dff}, num_hidden={hidden}, risk={risk}, ii={ii}: {e}")
                                pass


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/work/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended 

                       Metric  High aggressive  Aggressive  Balanced  \
0   Annualized Return (No RF)         0.407307    0.437414  0.441019   
1      Annualized Std (No RF)         0.420640    0.409404  0.410277   
2    Annualized Excess Return         0.391677    0.421451  0.425017   
3       Annualized Excess Std         0.420619    0.409391  0.410263   
4                Sharpe Ratio         0.941568    1.040948  1.047521   
5               Sortino Ratio         1.365533    1.503019  1.514480   
6           Arithmetic Return         0.341909    0.363107  0.365616   
7            Geometric Return         0.266694    0.299856  0.302633   
8            Maximum Drawdown         0.404755    0.400098  0.398336   
9         Annualized Skewness        -0.084104   -0.090467 -0.099511   
10        Annualized Kurtosis         8.351455    8.822579  8.788349   
11          Cumulative Return         1.567229    1.845937  1.870264   
12            Monthly 95% VaR         0.123015    0.123011  0.12

/home/work/DINN_COPY/utils/main_analysis.py:145: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = df['Cumulative Value'].resample('M').last().pct_change().dropna()
/home/work/DINN_COPY/utils/main_analysis.py:146: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_rf = (1 + df['rf']).resample('M').prod() - 1
/home/work/DINN_COPY/utils/main_analysis.py:145: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = df['Cumulative Value'].resample('M').last().pct_change().dropna()
/home/work/DINN_COPY/utils/main_analysis.py:146: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_rf = (1 + df['rf']).resample('M').prod() - 1
/home/work/DINN_COPY/utils/main_analysis.py:145: FutureWarning: 'M' is deprecated and will be removed in a future version, ple

['$0.4227 \\pm 0.0321$',
 '$0.4046 \\pm 0.0024$',
 '$1.0169 \\pm 0.0758$',
 '$1.4596 \\pm 0.1144$',
 '$0.3769 \\pm 0.0074$',
 '$0.1243 \\pm 0.0007$',
 '$0.1913 \\pm 0.0158$',
 '$2.9477 \\pm 0.2534$']

,Metric,HA,A,B,C,HC
0,Annualized Return (No RF),$0.4616 \pm 0.0171$,$0.4458 \pm 0.0165$,$0.4227 \pm 0.0321$,$0.3963 \pm 0.0111$,$0.3312 \pm 0.0104$
1,Annualized Std (No RF),$0.4208 \pm 0.0026$,$0.4105 \pm 0.0012$,$0.4046 \pm 0.0024$,$0.4005 \pm 0.0010$,$0.3826 \pm 0.0034$
2,Annualized Excess Return,$0.4454 \pm 0.0169$,$0.4297 \pm 0.0163$,$0.4069 \pm 0.0317$,$0.3808 \pm 0.0109$,$0.3164 \pm 0.0103$
3,Annualized Excess Std,$0.4207 \pm 0.0026$,$0.4104 \pm 0.0012$,$0.4046 \pm 0.0024$,$0.4005 \pm 0.0010$,$0.3825 \pm 0.0034$
4,Sharpe Ratio,$1.0704 \pm 0.0385$,$1.0586 \pm 0.0371$,$1.0169 \pm 0.0758$,$0.9614 \pm 0.0264$,$0.8364 \pm 0.0332$
5,Sortino Ratio,$1.5521 \pm 0.0485$,$1.5176 \pm 0.0590$,$1.4596 \pm 0.1144$,$1.3648 \pm 0.0400$,$1.1984 \pm 0.0546$
6,Arithmetic Return,$0.3798 \pm 0.0117$,$0.3689 \pm 0.0114$,$0.3526 \pm 0.0227$,$0.3340 \pm 0.0079$,$0.2862 \pm 0.0078$
7,Geometric Return,$0.3156 \pm 0.0148$,$0.3068 \pm 0.0143$,$0.2895 \pm 0.0274$,$0.2719 \pm 0.0097$,$0.2234 \pm 0.0110$
8,Maximum Drawdown,$0.3936 \pm 0.0010$,$0.3823 \pm 0.0039$,$0.3769 \pm 0.0074$,$0.3828 \pm 0.0055$,$0.3406 \pm 0.0157$
9,Annualized Skewness,$-0.0850 \pm 0.0212$,$-0.1099 \pm 0.0092$,$-0.1420 \pm 0.0239$,$-0.1911 \pm 0.0114$,$-0.2438 \pm 0.0101$
